In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import transformers

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


# Data Exploration

In [2]:
df = pd.read_csv("../Static Data/indeed.csv")
df.head()

,Job Title,Job Description,Job Type,Categories,Location,City,State,Country,Zip Code,Address,...,Employer Phone,Employer Logo,Companydescription,Employer Location,Employer City,Employer State,Employer Country,Employer Zip Code,Uniq Id,Crawl Timestamp
0,Shift Manager,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Mission Hills, CA 91345",Mission Hills,CA,United States,91345,NaN,...,NaN,https://d2q79iu7y748jz.cloudfront.net/s/_squar...,Del Taco is an American quick service restaura...,"Mission Hills, CA 91345",Mission Hills,CA,United States,91345,511f9a53920f4641d701d51d3589349f,2019-08-24 09:13:18 +0000
1,Operations Support Manager,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Atlanta, GA 30342",Atlanta,GA,United States,30342,NaN,...,NaN,https://d2q79iu7y748jz.cloudfront.net/s/_logo/...,"Based in Atlanta, FOCUS Brands Inc. is an inno...",NaN,NaN,NaN,United States,NaN,4955daf0a3facbe2acb6c429ba394e6d,2019-09-19 08:16:55 +0000
2,Senior Product Manager - Data,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Chicago, IL",Chicago,IL,United States,NaN,NaN,...,NaN,NaN,Vibes Corp. reputation was built and establish...,NaN,NaN,NaN,United States,NaN,a0e0d12df1571962b785f17f43ceae12,2019-09-18 02:13:10 +0000
3,Part-Time Office Concierge,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Festus, MO",Festus,MO,United States,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,United States,NaN,56e411fd731f76ac916bf4fb169250e9,2019-10-24 16:39:13 +0000
4,Print & Marketing Associate,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Cedar Rapids, IA 52404",Cedar Rapids,IA,United States,52404,NaN,...,NaN,https://d2q79iu7y748jz.cloudfront.net/s/_logo/...,"Staples is The Worklife Fulfillment Company, h...","Cedar Rapids, IA 52404",Cedar Rapids,IA,United States,52404,3fff5c0ad6981bf4bff6260bd5feab63,2019-08-24 22:29:10 +0000


In [3]:
df.shape

(30002, 30)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30002 entries, 0 to 30001
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Job Title           30002 non-null  object 
 1   Job Description     30002 non-null  object 
 2   Job Type            0 non-null      float64
 3   Categories          0 non-null      float64
 4   Location            30002 non-null  object 
 5   City                30002 non-null  object 
 6   State               30002 non-null  object 
 7   Country             30002 non-null  object 
 8   Zip Code            16252 non-null  object 
 9   Address             0 non-null      float64
 10  Salary From         0 non-null      float64
 11  Salary To           0 non-null      float64
 12  Salary Period       0 non-null      float64
 13  Apply Url           18392 non-null  object 
 14  Apply Email         0 non-null      float64
 15  Employees           0 non-null      float64
 16  Indu

# Get Text Data

In [5]:
df_text = df[['Job Title', 'Job Description']]
df_text.head()

,Job Title,Job Description
0,Shift Manager,"<div id=""jobDescriptionText"" class=""jobsearch-..."
1,Operations Support Manager,"<div id=""jobDescriptionText"" class=""jobsearch-..."
2,Senior Product Manager - Data,"<div id=""jobDescriptionText"" class=""jobsearch-..."
3,Part-Time Office Concierge,"<div id=""jobDescriptionText"" class=""jobsearch-..."
4,Print & Marketing Associate,"<div id=""jobDescriptionText"" class=""jobsearch-..."


In [6]:
# process Job Description
def process_html(txt):
    result = re.sub(".*?<.*?>","",txt)
    return result

In [7]:
print(process_html(df_text['Job Description'][0]))

WE ARE LOOKING FOR TOP PERFORMERS TO GROW WITH US!

Del Taco needs people with the ability to handle multiple responsibilities and deliver the quality and guest service that defines our brand. While we look for you to bring the right skills and personality for a Shift Manager position, we'll also provide the training and support you need to continue to grow and advance.

Our Shift Managers support our stores and General Managers in creating an amazing guest experience, and empowering their team to be at their very best. Our Shift Managers create a culture in our restaurants that is exciting, optimistic and rewarding.

Del Taco Shift Managers have significant opportunities for growth and career advancement and are charged with supporting the General Manager with the tasks of training the next generation of leaders, guiding them to be promoted and empowering them to have a successful career.


Supports General Manager in supervising team that delivers effective results in the areas of qu

# Classification Model

In [8]:
from transformers import (
    TokenClassificationPipeline,
    AutoModelForTokenClassification,
    AutoTokenizer,
)
from transformers.pipelines import AggregationStrategy
import numpy as np

# Define keyphrase extraction pipeline
class KeyphraseExtractionPipeline(TokenClassificationPipeline):
    def __init__(self, model, *args, **kwargs):
        super().__init__(
            model=AutoModelForTokenClassification.from_pretrained(model),
            tokenizer=AutoTokenizer.from_pretrained(model),
            *args,
            **kwargs
        )

    def postprocess(self, all_outputs):
        results = super().postprocess(
            all_outputs=all_outputs,
            aggregation_strategy=AggregationStrategy.SIMPLE,
        )
        return np.unique([result.get("word").strip() for result in results])

# Load pipeline
model_name = "ml6team/keyphrase-extraction-kbir-inspec"
extractor = KeyphraseExtractionPipeline(model=model_name)

ImportError: 
AutoModelForTokenClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [ ]:
print(transformers.__version__)

2.1.1


In [9]:
! pip install torch

Defaulting to user installation because normal site-packages is not writeable
                                              0.0/192.3 MB ? eta -:--:--
                                              0.1/192.3 MB 2.6 MB/s eta 0:01:13
                                              0.2/192.3 MB 1.8 MB/s eta 0:01:45
                                              0.3/192.3 MB 2.1 MB/s eta 0:01:31
                                              0.3/192.3 MB 1.8 MB/s eta 0:01:48
                                              0.4/192.3 MB 1.8 MB/s eta 0:01:47
                                              0.5/192.3 MB 1.7 MB/s eta 0:01:54
                                              0.5/192.3 MB 1.6 MB/s eta 0:02:01
                                              0.6/192.3 MB 1.5 MB/s eta 0:02:11
                                              0.7/192.3 MB 1.5 MB/s eta 0:02:06
                                              0.7/192.3 MB 1.4 MB/s eta 0:02:18
                                              0.8

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
